## Pseudo-Random Number Generation in Simulation

Make sure you are able to the run the following cell. It'd prompt errors if you are missing the needed packages before you continue with this tutorial. If that happens, you need to install the missing packages and restart the notebook.

In [ ]:
# for random distributions, random number generators, statistics
import random
import numpy as np
import scipy.stats as stats

# the simulation
import simulus

# for data visualization
import matplotlib.pyplot as plt
%matplotlib inline

# for animation inside the notebook
from ipywidgets import interact
import ipywidgets as widgets


For this tutorial, we will use the `scipy.stats` module for random number generation. Python's  native `random` module can also be used for this purpose. In fact, `simulus` uses the `random` module as its core random number generator. But the `scipy.stats` module contains a huge number of probability distributions (more than 120 of them) and many useful statistical functions, which would be be extremely useful for any simulation tasks in general, and of course, would be more than sufficient for our tutorial.

The `scipy.stats` module uses the pseudo-random number generator provided by `numpy`. Both `numpy` and Python's `random` module use the Mersenne Twister as the core generator. This generator produces 53-bit precision floats and has a period of $2^{19937}-1$ and very good random properties.

### Random Variables in `scipy.stats`

The `scipy.stats` module contains a large number of probability distributions. Let's first examine a few. One of the most common distribution used in simulation is the exponential distribution. The function `expon(scale)` creates an exponential random variable with the parameter 'scale' being the mean of the distribution. 

In [ ]:
x = stats.expon(scale=2)
print(x)

We can eaily find the mean, the median, and the standard deviation of the random variable:

In [ ]:
x.mean(), x.median(), x.std()

We can also use the `stats(moments)` method to find the mean(‘m’), variance (‘v’), skew (‘s’), and kurtosis(‘k’), where 'moments' specifies which we would like to have. In the following example, we want all of them:    

In [ ]:
m, v, s, k = x.stats(moments='mvsk')
print('mean=%g, var=%g, skew=%g, kurtosis=%g' % (m, v, s, k))

We can also plot the probability density function (pdf) and the cumulative density function (cdf) of the random variable. In the following, we first define a couple of functions for plotting:

In [ ]:
# a generic function for plotting a random variable
def plot_rv(rv, title, xmin=None, xmax=None):
    # find the range we'd like to draw (ppf() is the inverse cdf)
    if xmin is None:
        xmin = rv.ppf(0.01)
    if xmax is None:
        xmax = rv.ppf(0.99)

    # get the data points for pdf and cdf
    xs = np.linspace(xmin, xmax, 100)
    ys = rv.pdf(xs)
    yys = rv.cdf(xs)
    
    plt.fill_between(xs, ys, color='#7fc97f', alpha=0.7)
    plt.plot(xs, ys, color='#7fc97f', lw=3, alpha=0.9, label='pdf')

    plt.fill_between(xs, yys, color='#beaed4', alpha=0.7)
    plt.plot(xs, yys, color='#beaed4', lw=3, alpha=0.9, label='cdf')
    
    plt.title(title)
    plt.xlim(xmin, xmax)
    plt.ylim(0)
    plt.legend()
    plt.show()
    

In [ ]:
# create an exponentially distributed random variable with 
# the given scale (mean) and use the above function to plot it 
def plot_expon(scale):
    rv = stats.expon(scale=scale)
    plot_rv(rv, "exponential (scale=%g)" % scale)

Now we are ready to plot an exponential distribution:

In [ ]:
plot_expon(2.0)

We can also dynamically change the 'scale' parameter (using a slider) and plot the distribution on the fly. In the following, move the slider to change the scale value.

In [ ]:
slider = widgets.FloatSlider(min=0.1, max=5, value=2)
interact(plot_expon, scale=slider)
None

Let's do the same for two other distributions just for fun:

In [ ]:
def plot_gamma(a, scale):
    rv = stats.gamma(a=a, scale=scale)
    plot_rv(rv, "gamma (a=%g, scale=%g)" % (a,scale))

slider_a = widgets.FloatSlider(min=0.1, max=5, value=2.5)
slider_scale = widgets.FloatSlider(min=0.1, max=5, value=0.4)
interact(plot_gamma, a=slider_a, scale=slider_scale)
None

In [ ]:
def plot_norm(loc, scale):
    rv = stats.norm(loc=loc, scale=scale)
    plot_rv(rv, "normal (loc=%g, scale=%g)" % (loc, scale))
    
slider_loc = widgets.FloatSlider(min=-2, max=2, value=0)
slider_scale = widgets.FloatSlider(min=0.1, max=2, value=1)
interact(plot_norm, loc=slider_loc, scale=slider_scale)
None

### Generating Random Variates

Once a random variable is created, one can use the `rvs(size)` method to draw random samples from the given distribution, where the 'size' argument specifies the number of samples. To get repeatable results, we should first set the random seed. Recall that the scipy.stats module uses the pseudo-random number generator provided by the `numpy` module. Therefore we set the random seed using the `numpy.random.seed()` function.

In [ ]:
x = stats.expon(scale=2.0)

# get the first 1000 random samples and show the first 3 
np.random.seed(13579)
xs1 = x.rvs(1000)
print('the first 1000 samples: %r...' % xs1[:3])

# get another 1000 random samples and show the first 3
np.random.seed(24680)
xs2 = x.rvs(1000)
print('the second 1000 samples: %r...' % xs2[:3])

In [ ]:
# if we reset the random seed, we should get the same sequence
np.random.seed(13579)
xs1 = x.rvs(1000)
print('repeat the first 1000 samples: %r...' % xs1[:3])

np.random.seed(24680)
xs2 = x.rvs(1000)
print('repeat the second 1000 samples: %r...' % xs2[:3])

To make sure that the random number are indeed drawn from the expected random distribution, we can plot the histogram of the samples and compare that with the true distribution.

In [ ]:
plt.figure(figsize=(10,5))

xmin, xmax = rv.ppf(0.001), rv.ppf(0.999)
xs = np.linspace(xmin, xmax, 100)
ys = x.pdf(xs)

axs = plt.subplot(1, 2, 1)
axs.hist(xs1, alpha=0.5, bins='auto', density=True)
axs.plot(xs, ys, 'r-')
axs.set_xlim(xmin, xmax)
axs.set_title("histogram of xs1")

axs = plt.subplot(1, 2, 2)
axs.hist(xs2, alpha=0.5, bins='auto', density=True)
axs.plot(xs, ys, 'r-')
axs.set_xlim(xmin, xmax)
axs.set_title("histogram of xs2")

plt.show()

### Repeatable Random Sequences for Simulation

In [ ]:
random.seed(13597)

sim = simulus.simulator('prng')
randstreams = [sim.rng().randrange(2**32) for _ in range(10)]
print(randstreams)

In [ ]:
def plot_rv_samples(rv, title, xmin=None, xmax=None, samples=1000):
    m = rv.mean()
    sd = rv.std()
    if xmin is None: xmin = m-4*sd
    if xmax is None: xmax = m+4*sd
    xs = np.linspace(xmin, xmax, 100)
    ys = rv.pdf(xs)
    yys = rv.cdf(xs)
    plt.fill_between(xs, ys, label="pdf", color='#7fc97f', alpha=0.7)
    plt.fill_between(xs, yys, label="cdf", color='#beaed4', alpha=0.7)
    plt.vlines(m, ymin=0, ymax=rv.cdf(m), color='k', linestyle='--')
    plt.hlines(rv.cdf(m), xmin=xmin, xmax=m, color='k', linestyle='--')
    plt.text(m, rv.cdf(m), 'mean=%g\nsd=%g'% (m,sd), fontsize=12)
    plt.legend()
    v = rv.rvs(samples)
    plt.hist(v, density=True, histtype='stepfilled', alpha=0.4, bins=20)
    plt.title(title)
    plt.xlim(xmin, xmax)
    plt.ylim(0)
    plt.show()

In [ ]:
def plot_expon_samples(scale, samples=1000):
    rv = stats.expon(scale=scale)
    rv.random_state = seed=np.random.RandomState(randstreams[0])
    plot_rv_samples(rv, "exponential (scale=%g)" % scale, xmin=0, samples=samples)
    
plot_expon_samples(1.2)

In [ ]:
slider_scale = widgets.FloatSlider(min=0.1, max=5, value=1.2)
slider_samples = widgets.IntSlider(min=10, max=1000, value=100)
interact(plot_expon_samples, scale=slider_scale, samples=slider_samples)
None

In [ ]:
plot_expon_samples(1.6, 118)

In [ ]:
rv2 = stats.norm(scale=1.2)
rv2.random_state = np.random.RandomState(randstreams[1])
y = rv2.rvs(1000)
y.mean()

In [ ]:
plt.scatter(x,y)